In [42]:
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common import simple_sbml
from SBMLLint.common.reaction import Reaction
from SBMLLint.tools import sbmllint
from SBMLLint.tools import print_reactions

import os
import numpy as np
import pandas as pd
from pulp import *
import matplotlib.pyplot as plt
import time

from scipy.linalg import lu, inv
from scipy.optimize import linprog

In [43]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix

from SBMLLint.games.som import SOM
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

In [8]:
imsc_filepath = os.path.join("/Users/woosubs/Desktop/ModelEngineering/dummy/dummy_data/imsc1255.xml")
simple = SimpleSBML()
simple.initialize(imsc_filepath)

In [31]:
# MODEL = "BIOMD0000000167.xml"
# simple = SimpleSBML()
# simple.initialize(os.path.join(cn.BIOMODELS_DIR, MODEL))

In [67]:
s = StoichiometryMatrix(simple)
mat = s.stoichiometry_matrix
# create a dummy matrix
mat = pd.DataFrame(np.array([[-1, 1, 1, 0, 0],
                             [-1, 1, 0, 0, 0],
                             [-1, 0, 1, 0, 0],
                            [0, 0, 0, -1, 1]]),
                   columns=['A', 'B','C', 'D', 'E'], index=['R1','R2','R3', 'R4']).T

mat

,R1,R2,R3,R4
A,-1,-1,-1,0
B,1,1,0,0
C,1,0,1,0
D,0,0,0,-1
E,0,0,0,1


In [68]:
s.isConsistent()

False

In [69]:
def solveMILP(mat):
  prob = LpProblem("Finding_Unconserved_Metabolites", LpMaximize)
  species = list(mat.index)
  species_inclusion = pulp.LpVariable.dicts("species", species, cat="Binary")
  species_mass = pulp.LpVariable.dicts("mass", species, cat="Continuous")
  # objective function (to maximize the number of species)
  prob += lpSum([species_inclusion[i] for i in species])
  # constraint 1 (for each reaction, the sum(stoichiometry[i]*mass[i])=0)
  for reaction in mat.columns:
    prob += lpSum(sum([mat[reaction][species]*species_mass[species] for species in species_inclusion])) == 0
  # constraint 2 (species_inclusion is less than or equal to mass of each species)
  for species in species_mass.keys():
    prob += species_inclusion[species] <= species_mass[species]
  prob.solve()
  return prob
def getUnconservedMetabolites(milp_result):
  unconserved_metabolites = []
  for v in milp_result.variables():
    if v.varValue==0:
      if v.name[:7]=="species":
        unconserved_metabolites.append(v.name[8:])
  return unconserved_metabolites
#
milp_start = time.time()
milp_result = solveMILP(mat)
unconserved_metabolites = getUnconservedMetabolites(milp_result)
milp_end = time.time()
milp_time = milp_end - milp_start
print("MILP time: %f" % milp_time)

MILP time: 0.017105


In [70]:
for v in milp_result.variables():
    print(v.varValue)
    print(v.name)

0.0
mass_A
0.0
mass_B
0.0
mass_C
1.0
mass_D
1.0
mass_E
0.0
species_A
0.0
species_B
0.0
species_C
1.0
species_D
1.0
species_E


In [66]:
milp_result

Finding_Unconserved_Metabolites:
MAXIMIZE
1*species_A + 1*species_B + 1*species_C + 0
SUBJECT TO
_C1: - mass_A + mass_B + mass_C = 0

_C2: - mass_A + mass_B = 0

_C3: - mass_A + mass_C = 0

_C4: - mass_A + species_A <= 0

_C5: - mass_B + species_B <= 0

_C6: - mass_C + species_C <= 0

VARIABLES
mass_A free Continuous
mass_B free Continuous
mass_C free Continuous
0 <= species_A <= 1 Integer
0 <= species_B <= 1 Integer
0 <= species_C <= 1 Integer

In [41]:
# milp_result

In [55]:
len(unconserved_metabolites)

1

In [22]:
boundary_elements = []
for r in simple.reactions:
  if r.category==cn.REACTION_BOUNDARY:
    for re in r.reactants:
      boundary_elements.append(re.molecule.name)
    for re in r.products:
      boundary_elements.append(re.molecule.name)
res = list(set(boundary_elements))
print(res)

['M_n2m2nmasn_e', 'M_na1_e', 'M_4mtolbutamide_e', 'M_tchola_e', 'M_dopa_e', 'M_gthox_e', 'M_sucr_e', 'M_thm_e', 'M_24nph_e', 'M_estrones_e', 'M_estradiolglc_e', 'M_o2_e', 'M_leu_DASH_DASH_DASH_L_e', 'M_pyr_e', 'M_dgchol_e', 'M_gln_DASH_DASH_DASH_L_e', 'M_tmndnc_e', 'M_btn_e', 'M_mercplaccys_e', 'M_dcsptn1_e', 'M_elaid_e', 'M_tettet6_e', 'M_phe_DASH_DASH_DASH_L_e', 'M_ethamp_r', 'M_25hvitd3_e', 'M_xoltri24_e', 'M_cl_e', 'M_thymd_e', 'M_ppa_e', 'M_tyr_DASH_DASH_DASH_L_e', 'M_no_e', 'M_strdnc_e', 'M_imp_e', 'M_his_DASH_DASH_DASH_L_e', 'M_thmtp_e', 'M_tcynt_e', 'M_for_e', 'M_nh4_e', 'M_34dhoxpeg_e', 'M_appnn_e', 'M_Lcystin_e', 'M_inost_e', 'M_camp_e', 'M_hdcea_e', 'M_pnto_DASH_DASH_DASH_R_e', 'M_10fthf7glu_e', 'M_h2o2_e', 'M_crvnc_e', 'M_ribflv_e', 'M_ttdca_e', 'M_retnglc_e', 'M_cmp_e', 'M_hcoumarin_e', 'M_fe2_e', 'M_dlnlcg_e', 'M_carveol_e', 'M_lnlnca_e', 'M_ncam_e', 'M_ps_DASH_hs_e', 'M_acac_e', 'M_utp_e', 'M_glu_DASH_DASH_DASH_L_e', 'M_ser_DASH_DASH_DASH_L_e', 'M_meoh_e', 'M_tetpent6_e'

In [23]:
len(res)

204

In [27]:
len(simple.molecules)

1850